# Gensim, word2vec

Gensim - вообще библиотека для тематического моделирования текстов. Один из компонентов в ней - реализация на python алгоритмов из библиотеки word2vec (которая в оригинале была написана на C++).

Если gensim у вас не стоит, то ставим:

`pip install gensim`

Поскольку иногда тренировка модели занимает много времени, то можно ещё вести лог событий.

In [3]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\ancatmara\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Процесс тренировки модели 

На вход модели даем текстовый файл, каждое предложение на отдельной строчке. 

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

Возьмем, например, "Бедную Лизу". В этом тексте заранее убрана пунтуация и слова приведены к нижнему регистру, но не лемматизированы и не размечены по частям речи.

In [15]:
f = 'liza.txt'
data = gensim.models.word2vec.LineSentence(f)

Инициализируем модель. Параметры в скобочках:
* data - данные, 
* size - размер вектора, 
* window - размер окна наблюдения,
* min_count - мин. частотность слова в корпусе, которое мы берем,
* sg - используемый алгоритм обучение (0 - CBOW, 1 - Skip-gram))

In [16]:
%time model = gensim.models.Word2Vec(data, size=500, window=10, min_count=2, sg=0)

2018-05-28 09:58:55,047 : INFO : collecting all words and their counts
2018-05-28 09:58:55,049 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-28 09:58:55,058 : INFO : collected 2025 word types from a corpus of 5045 raw words and 395 sentences
2018-05-28 09:58:55,059 : INFO : Loading a fresh vocabulary
2018-05-28 09:58:55,064 : INFO : min_count=2 retains 609 unique words (30% of original 2025, drops 1416)
2018-05-28 09:58:55,066 : INFO : min_count=2 leaves 3629 word corpus (71% of original 5045, drops 1416)
2018-05-28 09:58:55,071 : INFO : deleting the raw counts dictionary of 2025 items
2018-05-28 09:58:55,073 : INFO : sample=0.001 downsamples 63 most-common words
2018-05-28 09:58:55,076 : INFO : downsampling leaves estimated 2611 word corpus (72.0% of prior 3629)
2018-05-28 09:58:55,080 : INFO : estimated required memory for 609 words and 500 dimensions: 2740500 bytes
2018-05-28 09:58:55,085 : INFO : resetting layer weights
2018-05-28 09:58:55,107 :

Wall time: 255 ms


Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать.

In [17]:
model.init_sims(replace=True)

2018-05-28 09:59:00,403 : INFO : precomputing L2-norms of word weight vectors


Смотрим, сколько в модели слов:

In [18]:
print(len(model.wv.vocab))

609


In [22]:
print([w for w in model.wv.vocab])

['может', 'быть', 'никто', 'из', 'в', 'москве', 'не', 'знает', 'так', 'хорошо', 'города', 'сего', 'как', 'я', 'потому', 'что', 'чаще', 'моего', 'бывает', 'поле', 'более', 'без', 'куда', 'глаза', 'по', 'лугам', 'и', 'новые', 'места', 'или', 'красоты', 'но', 'всего', 'для', 'меня', 'то', 'место', 'мрачные', 'синова', 'монастыря', 'на', 'сей', 'горе', 'стороне', 'почти', 'всю', 'москву', 'сию', 'которая', 'великолепная', 'картина', 'когда', 'нее', 'солнце', 'лучи', 'его', 'бесчисленных', 'к', 'небу', 'луга', 'а', 'за', 'ними', 'под', 'которые', 'от', 'российской', 'хлебом', 'другой', 'реки', 'подле', 'там', 'тению', 'песни', 'тем', 'дни', 'столь', 'них', 'древних', 'еще', 'далее', 'же', 'с', 'своим', 'часто', 'прихожу', 'сие', 'всегда', 'вместе', 'страшно', 'между', 'травою', 'келий', 'опершись', 'стону', 'времен', 'которого', 'сердце', 'мое', 'иногда', 'себе', 'тех', 'здесь', 'перед', 'о', 'своих', 'ибо', 'все', 'него', 'жизни', 'чувства', 'кроме', 'лицом', 'взором', 'сквозь', 'видит', '

И сохраняем!

In [23]:
model.save('my.model')

2018-05-28 10:01:53,394 : INFO : saving Word2Vec object under my.model, separately None
2018-05-28 10:01:53,399 : INFO : not storing attribute vectors_norm
2018-05-28 10:01:53,401 : INFO : not storing attribute cum_table
2018-05-28 10:01:53,439 : INFO : saved my.model


## Работа с моделью

Для каких-то своих индивидуальных нужд и экспериментов бывает полезно самому натренировать модель на нужных данных и с нужными параметрами. Но для каких-то общих целей модели уже есть как для русского языка, так и для английского.

Модели для русского скачать можно здесь - http://rusvectores.org/ru/models

Скачаем модель для русского языка, созданную на основе НКРЯ. Поскольку модели бывают разных форматов, то для них написаны разные функции загрузки; бывает полезно учитывать это в своем скрипте:

In [5]:
import urllib.request

urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x21f277940f0>)

In [11]:
m = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'
if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.KeyedVectors.load(m)

2018-06-09 00:46:34,607 : INFO : loading projection weights from ruscorpora_mystem_cbow_300_2_2015.bin.gz
2018-06-09 00:46:48,478 : INFO : loaded (281776, 300) matrix from ruscorpora_mystem_cbow_300_2_2015.bin.gz


In [19]:
model.init_sims(replace=True)

2018-06-09 00:49:51,702 : INFO : precomputing L2-norms of word weight vectors


In [20]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'студент_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [21]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print(word + ' is not present in the model')

день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]
неделя_S 0.7165195941925049
месяц_S 0.6310489177703857
вечер_S 0.5828738808631897
утро_S 0.5676206946372986
час_S 0.5605547428131104
минута_S 0.5297019481658936
гекатомбеон_S 0.4897990822792053
денек_S 0.48224717378616333
полчаса_S 0.48217129707336426
ночь_S 0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S 0.6946247816085815
утро_S 0.5730193853378296
ноченька_S 0.5582467317581177
рассвет_S 0.5553582906723022
ночка_S 0.5351512432098389
полдень_S 0.5334426760673523
полночь_S 0.478694349527359
день_S 0.4780748784542084
сумерки_S 0.43902185559272766
фундерфун_S 0.4340824782848358


человек_S
[ 0.02013756 -0.02670704 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240307 -0.00025525 -0.14045683  0.04785006]
женщина_S 0.5979775190353394
парень_S 0.499

Находим косинусную близость пары слов:

In [22]:
print(model.similarity('человек_S', 'обезьяна_S'))

0.238956092849


Найди лишнее!

In [29]:
print(model.doesnt_match('яблоко_S груша_S виноград_S банан_S апельсин_S огурец_S'.split()))

огурец_S


Реши пропорцию!

In [33]:
print(model.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S'])[0][0])

пельмень_S


Это конечно все хорошо, но как понять, какая модель лучше? Или вот например я сделал свою модель, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Их два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. Подробнее читаем тут:  
* http://www.aclweb.org/aclwiki/index.php?title=Google_analogy_test_set_(State_of_the_art)
* https://www.aclweb.org/aclwiki/index.php?title=SimLex-999_(State_of_the_art)

Датасеты для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [ ]:
model.accuracy('ruanalogy_upos.txt', restrict_vocab=3000000)